In [1]:
# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# LGBMでやってみる。
# todo:bayseの方と、インプットの与え方をあわせる。今はOne-Hotが一致していない。
# 的中予想確率10%以上のうち、オッズで見合うものだけ買った場合のシミュレーションをする。

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import csv

In [2]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler


In [3]:
# 舟券の配列を取得
funakenList=[]
with open(os.environ['BR_HOME']+'/boatrace/config/3t_list.dat') as f:
    reader=csv.reader(f)
    for row in reader:
        funakenList.append(row)
funakenID = [i for i in range(120)]
funakenDict=dict(zip(funakenList[0],funakenID))
print(funakenDict)

{'1-2-3': 0, '1-2-4': 1, '1-2-5': 2, '1-2-6': 3, '1-3-2': 4, '1-3-4': 5, '1-3-5': 6, '1-3-6': 7, '1-4-2': 8, '1-4-3': 9, '1-4-5': 10, '1-4-6': 11, '1-5-2': 12, '1-5-3': 13, '1-5-4': 14, '1-5-6': 15, '1-6-2': 16, '1-6-3': 17, '1-6-4': 18, '1-6-5': 19, '2-1-3': 20, '2-1-4': 21, '2-1-5': 22, '2-1-6': 23, '2-3-1': 24, '2-3-4': 25, '2-3-5': 26, '2-3-6': 27, '2-4-1': 28, '2-4-3': 29, '2-4-5': 30, '2-4-6': 31, '2-5-1': 32, '2-5-3': 33, '2-5-4': 34, '2-5-6': 35, '2-6-1': 36, '2-6-3': 37, '2-6-4': 38, '2-6-5': 39, '3-1-2': 40, '3-1-4': 41, '3-1-5': 42, '3-1-6': 43, '3-2-1': 44, '3-2-4': 45, '3-2-5': 46, '3-2-6': 47, '3-4-1': 48, '3-4-2': 49, '3-4-5': 50, '3-4-6': 51, '3-5-1': 52, '3-5-2': 53, '3-5-4': 54, '3-5-6': 55, '3-6-1': 56, '3-6-2': 57, '3-6-4': 58, '3-6-5': 59, '4-1-2': 60, '4-1-3': 61, '4-1-5': 62, '4-1-6': 63, '4-2-1': 64, '4-2-3': 65, '4-2-5': 66, '4-2-6': 67, '4-3-1': 68, '4-3-2': 69, '4-3-5': 70, '4-3-6': 71, '4-5-1': 72, '4-5-2': 73, '4-5-3': 74, '4-5-6': 75, '4-6-1': 76, '4-6-2':

In [4]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20180101"
trainEndDate="20180731"
# test はtrainからsplitする

In [5]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [6]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))

traindata: 31824


In [7]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,2-1-3,1,0.3636,0.5152,0.3558,0.5092,A1,13.871549,0,0.4746,...,0.4545,0.5421,0.6789,B2,0.851669,30.9,2018-01-01,20180101-06-01,3,5.0
1,2-1-6,0,0.2462,0.4462,0.3466,0.4830,B1,3.409304,0,0.3636,...,0.4925,0.3392,0.5088,B1,0.886552,26.2,2018-01-01,20180101-06-02,3,5.0
2,3-5-4,0,0.3692,0.5231,0.3135,0.4108,B2,2.836839,0,0.2714,...,0.4091,0.3472,0.4870,B1,0.580764,99.2,2018-01-01,20180101-06-03,3,5.0
3,1-5-2,0,0.4308,0.6308,0.3676,0.5514,A2,7.488113,0,0.3636,...,0.3939,0.3176,0.4706,B1,0.583266,16.8,2018-01-01,20180101-06-04,3,5.0
4,4-1-3,0,0.3636,0.5152,0.4469,0.5754,B1,6.454111,0,0.3231,...,0.4925,0.3916,0.5663,B2,0.470489,51.5,2018-01-01,20180101-06-05,3,5.0


In [8]:
# 入力のデータ整形
#xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
#xdf.head()

# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()


,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,l2boat3r,...,l5score,l6Fcnt,l6boat2r,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,raceWaveHeight,raceWindSpeed
0,1,0.3636,0.5152,0.3558,0.5092,0,13.871549,0,0.4746,0.6610,...,0.827756,0,0.3182,0.4545,0.5421,0.6789,3,0.851669,3,5.0
1,0,0.2462,0.4462,0.3466,0.4830,2,3.409304,0,0.3636,0.5758,...,0.671987,0,0.3284,0.4925,0.3392,0.5088,2,0.886552,3,5.0
2,0,0.3692,0.5231,0.3135,0.4108,3,2.836839,0,0.2714,0.5000,...,4.669515,0,0.3030,0.4091,0.3472,0.4870,2,0.580764,3,5.0
3,0,0.4308,0.6308,0.3676,0.5514,1,7.488113,0,0.3636,0.5303,...,1.435647,0,0.2121,0.3939,0.3176,0.4706,2,0.583266,3,5.0
4,0,0.3636,0.5152,0.4469,0.5754,2,6.454111,0,0.3231,0.6154,...,1.682992,0,0.3582,0.4925,0.3916,0.5663,3,0.470489,3,5.0


In [9]:
# ファイルから作った辞書で変換する
ydf=df['funaken']
ydf=pd.DataFrame(ydf.replace(funakenDict))
print(ydf.head())


   funaken
0       20
1       23
2       54
3       12
4       61


In [10]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
print(odf)

[30.9 26.2 99.2 ...  5.1  7.9  5.7]


In [11]:
X_train, X_test, y_train, y_test,o_train,o_test = train_test_split(xdf, ydf,odf)

In [12]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [13]:
lgbm_params = {
    # 多値分類問題
    'objective': 'multiclass',
    # クラス数は 120
    'num_class': 120,
    #'class_weight':'balanced',
    'random_state':999,
    # 以下、ハイパーパラメタ
    'max_depth':3,
    'num_leaves':19,
    'min_data_in_leaf':300,
    # 正則化
    'reg_alpha':9.591,
    'reg_lambda':9.928 ,
}


In [14]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [15]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

[1]	valid_0's multi_logloss: 4.67019
[2]	valid_0's multi_logloss: 4.58449
[3]	valid_0's multi_logloss: 4.51648
[4]	valid_0's multi_logloss: 4.45964
[5]	valid_0's multi_logloss: 4.41087
[6]	valid_0's multi_logloss: 4.36974
[7]	valid_0's multi_logloss: 4.33295
[8]	valid_0's multi_logloss: 4.2999
[9]	valid_0's multi_logloss: 4.27105
[10]	valid_0's multi_logloss: 4.24467
[11]	valid_0's multi_logloss: 4.22075
[12]	valid_0's multi_logloss: 4.19956
[13]	valid_0's multi_logloss: 4.17996
[14]	valid_0's multi_logloss: 4.1621
[15]	valid_0's multi_logloss: 4.14557
[16]	valid_0's multi_logloss: 4.13032
[17]	valid_0's multi_logloss: 4.11617
[18]	valid_0's multi_logloss: 4.10355
[19]	valid_0's multi_logloss: 4.09113
[20]	valid_0's multi_logloss: 4.08045
[21]	valid_0's multi_logloss: 4.07019
[22]	valid_0's multi_logloss: 4.06066
[23]	valid_0's multi_logloss: 4.05191
[24]	valid_0's multi_logloss: 4.04354
[25]	valid_0's multi_logloss: 4.03563
[26]	valid_0's multi_logloss: 4.02829
[27]	valid_0's multi_lo

In [16]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

# 精度 (Accuracy) を計算する
accuracy = sum(y_test['funaken'] == y_pred_max) / len(y_test)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_test)):
    if (y_test.iloc[i])['funaken']==y_pred_max[i]:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        res += o_test[i] -1
    else:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        pass
print("resultReturn:",res/len(y_test))


accuracy: 0.08584715937657114
resultReturn: 0.6940925087983908


In [17]:
print(len(y_pred))
# オッズを見て判断する場合
resAmount=0
buyAmount=0

dfIndexList=y_test['funaken'].index
for i in range(len(y_test)):
    raceId = df.iloc[dfIndexList[i]]['raceId']
    #print(raceId)
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)
        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
    #print((loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] )
    #if ( (loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] > 1) :
    #    print("ok")
    #print(y_pred[i][0])
    #print(raceId)
    for j in range(120):
        if y_pred[i][j]> 0.10 and ( (y_pred[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0])
            buyAmount+=1
            if y_test['funaken'].iloc[i]==j:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
            else:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                pass
print("resultReturn:",resAmount/buyAmount)


7956
buy: 20180225-16-05 6 9 16.8
buy: 20180127-01-01 9 1 16.5
buy: 20180719-23-02 63 2 7.5
buy: 20180719-06-07 90 6 23.4
buy: 20180512-07-08 197 2 13.1
buy: 20180407-18-03 224 0 20.1
buy: 20180225-21-06 241 8 26.9
buy: 20180724-14-07 245 0 10.5
☆hit!☆: 20180724-14-07 0 10.5
buy: 20180131-16-01 331 4 22.7
buy: 20180621-10-08 358 1 15.1
buy: 20180719-24-01 364 0 11.3
buy: 20180509-05-02 369 0 12.0
buy: 20180724-20-04 388 8 11.3
buy: 20180430-18-01 404 4 24.3
buy: 20180716-01-06 422 0 11.6
☆hit!☆: 20180716-01-06 0 11.6
buy: 20180128-06-06 462 0 10.7
buy: 20180128-06-06 462 1 30.5
buy: 20180302-20-02 508 1 15.4
buy: 20180602-24-06 639 0 16.4
buy: 20180104-12-07 683 20 18.7
buy: 20180316-24-05 684 1 14.6
buy: 20180405-07-06 687 0 12.5
buy: 20180622-19-10 692 0 15.7
buy: 20180614-21-11 699 3 16.1
buy: 20180420-11-05 710 2 13.0
buy: 20180129-05-07 726 0 18.7
buy: 20180126-17-08 730 0 13.8
buy: 20180706-24-04 781 0 13.3
buy: 20180412-09-05 867 0 14.8
buy: 20180501-05-09 873 0 10.5
buy: 201805

buy: 20180614-09-08 5813 1 34.5
buy: 20180528-17-03 5835 3 12.3
buy: 20180530-17-03 5884 0 11.2
buy: 20180115-11-08 5892 6 15.2
buy: 20180415-24-01 5908 4 13.9
buy: 20180303-21-08 5952 0 15.5
buy: 20180515-10-02 5973 8 13.5
buy: 20180401-12-05 6016 7 11.6
buy: 20180715-21-04 6063 0 15.5
buy: 20180715-21-04 6063 4 13.5
buy: 20180423-02-06 6074 12 15.8
buy: 20180506-16-01 6114 1 10.1
buy: 20180515-20-09 6117 0 20.9
buy: 20180208-01-06 6127 1 15.3
buy: 20180523-08-06 6202 0 20.0
buy: 20180505-15-10 6220 1 15.7
buy: 20180302-24-01 6323 0 15.3
buy: 20180111-16-11 6358 11 21.4
buy: 20180510-21-04 6492 4 14.5
buy: 20180501-15-06 6532 0 13.8
☆hit!☆: 20180501-15-06 0 13.8
buy: 20180106-16-08 6575 5 14.4
buy: 20180501-06-05 6587 12 15.5
buy: 20180610-06-05 6632 0 15.0
buy: 20180624-06-05 6703 23 19.0
buy: 20180129-13-01 6730 12 16.3
buy: 20180430-01-08 6804 0 23.6
buy: 20180413-10-03 6830 4 11.8
☆hit!☆: 20180413-10-03 4 11.8
buy: 20180720-24-12 6831 2 12.3
buy: 20180401-21-01 6897 8 15.0
buy: 20

In [18]:
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )

totalRace,buy,return 7956 311 232.20000000000002


In [19]:
# trainの回収率を計算
y_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

#print(X_train.head())
#print(y_train[0:5])
#print(y_pred_max[0:5])
#c = collections.Counter(y_pred_max)
#print(len(c) )

# 精度 (Accuracy) を計算する
accuracy = sum(y_train == y_pred_max) / len(y_train)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_train)):
    if y_train.iloc[i]==y_pred_max[i]:
        res += o_train[i] -1
    else:
        pass
print("return:",res/len(y_train))


ValueError: Unable to coerce to Series, length must be 1: given 23868

In [ ]:
print(y_pred[0])

In [ ]:
print(df.iloc[X_train.index]['raceId'])

In [ ]:
print(model.feature_importance())